In [34]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [35]:
data = pd.read_csv('./data.csv')

In [36]:
data['Mood'] = ''

In [37]:
for i in range(len(data)):
    if data.at[i, 'label'] <= 0.25:
        data.at[i, 'Mood'] = 0
    if data.at[i, 'label'] > 0.25 and data.at[i, 'label'] <= 0.5:
        data.at[i, 'Mood'] = 1
    if data.at[i, 'label'] > 0.5 and data.at[i, 'label'] <= 0.75:
        data.at[i, 'Mood'] = 2
    if data.at[i, 'label'] > 0.75:
        data.at[i, 'Mood'] = 3

data['Mood'] = data['Mood'].astype(int)

In [38]:
data.head()

,seq,label,Mood
0,"No, no\r\nI ain't ever trapped out the bando\r...",0.626,2
1,"The drinks go down and smoke goes up, I feel m...",0.630,2
2,She don't live on planet Earth no more\r\nShe ...,0.240,0
3,"Trippin' off that Grigio, mobbin', lights low\...",0.536,2
4,"I see a midnight panther, so gallant and so br...",0.371,1


In [10]:
import re
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [11]:
x = []
sentences = list(data["seq"])
for sen in sentences:
    x.append(preprocess_text(sen))

In [12]:
labels = data['Mood']
y = labels.values

In [13]:
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)
# x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

# X_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [14]:
x = pad_sequences(x, padding='post', maxlen=maxlen)

In [15]:
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
dtr = DecisionTreeClassifier()

In [19]:
dtr.fit(x_train, y_train)

DecisionTreeClassifier()

In [21]:
from sklearn.metrics import classification_report

In [23]:
print(classification_report(y_test, dtr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.32      0.35      0.34      9995
           1       0.34      0.33      0.34     14866
           2       0.32      0.31      0.32     13724
           3       0.25      0.25      0.25      8921

    accuracy                           0.32     47506
   macro avg       0.31      0.31      0.31     47506
weighted avg       0.32      0.32      0.32     47506



In [24]:
from sklearn.svm import SVC

In [25]:
svc = SVC()

In [26]:
svc.fit(x_train, y_train)

SVC()

In [27]:
print(classification_report(y_test, svc.predict(x_test)))

              precision    recall  f1-score   support

           0       0.42      0.24      0.30      9995
           1       0.33      0.54      0.41     14866
           2       0.34      0.42      0.38     13724
           3       0.56      0.03      0.06      8921

    accuracy                           0.35     47506
   macro avg       0.41      0.31      0.29     47506
weighted avg       0.39      0.35      0.31     47506



In [28]:
import xgboost as xgb

In [29]:
clsfr = xgb.XGBClassifier()

In [30]:
clsfr.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [31]:
print(classification_report(y_test, clsfr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.42      0.28      0.34      9995
           1       0.33      0.49      0.40     14866
           2       0.34      0.40      0.36     13724
           3       0.36      0.10      0.16      8921

    accuracy                           0.35     47506
   macro avg       0.36      0.32      0.31     47506
weighted avg       0.36      0.35      0.33     47506



In [33]:
import pickle as pkl
with open("xgboost.pkl", "wb") as file:
    pkl.dump(clsfr, file)

In [39]:
from sklearn.tree import DecisionTreeRegressor

In [40]:
dtreg = DecisionTreeRegressor()

In [41]:
dtreg.fit(x, data['label'])

DecisionTreeRegressor()

In [43]:
dtreg.predict([x_train[0]])

array([0.503])

In [46]:
from sklearn.metrics import mean_absolute_error

In [47]:
mean_absolute_error(y_test, dtreg.predict(x_test))

1.043651749968174

In [48]:
preds = dtreg.predict(x)

In [49]:
pd.DataFrame(preds).describe()

,0
count,158353.000000
mean,0.491052
std,0.238137
min,0.000000
25%,0.298000
50%,0.480000
75%,0.677000
max,0.998000


In [50]:
with open("dvalence.pkl", "wb") as file:
    pkl.dump(dtreg, file)